In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
stopwords=list(ENGLISH_STOP_WORDS)
stopwords

['herein',
 'whenever',
 'fifty',
 'your',
 'two',
 'further',
 'front',
 'to',
 'give',
 'fifteen',
 'fill',
 'thick',
 'nobody',
 'around',
 'either',
 'anywhere',
 'although',
 'over',
 'as',
 'last',
 'somehow',
 'so',
 'beyond',
 'interest',
 'out',
 'get',
 'rather',
 'hereafter',
 'than',
 'throughout',
 'mill',
 'into',
 'someone',
 'but',
 'what',
 'in',
 'between',
 'my',
 'due',
 'more',
 'sometime',
 'afterwards',
 'please',
 'such',
 'above',
 'perhaps',
 'now',
 'being',
 'via',
 'sixty',
 'hereby',
 'when',
 'here',
 'still',
 'moreover',
 'cannot',
 'under',
 'anyone',
 'has',
 'five',
 'some',
 'only',
 'against',
 'somewhere',
 'four',
 'thereby',
 'twelve',
 'was',
 'them',
 'much',
 'whereas',
 'we',
 'top',
 'thence',
 'have',
 'elsewhere',
 'whither',
 'twenty',
 'were',
 'co',
 'hereupon',
 'well',
 'hasnt',
 'a',
 'made',
 'alone',
 'upon',
 'before',
 'been',
 'could',
 'how',
 'because',
 'ie',
 'both',
 'an',
 'done',
 'nine',
 'thereupon',
 'another',
 'full

In [3]:
news=pd.read_csv("C:\\Users\khush\\OneDrive\\Desktop\\Personal Projects\\Fake News Prediction\\news.csv")

In [4]:
news

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
print("Rows:",news.shape[0])
print("Columns:",news.shape[1])

Rows: 20800
Columns: 5


In [6]:
news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
news=news.fillna(' ')
news.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

### We will combine the 'author' and 'title' column and use that new column (along with label) as the input for training the model. The other columns will not affect the functionality or performance of our model.

In [8]:
news['content']=news['author']+' '+news['title']
news['content']

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object

In [9]:
news=news.drop(['id','text'],axis=1)
temp=news.pop('content')
news.insert(2,'content',temp)
news

,title,author,content,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,Darrell Lucus House Dem Aide: We Didn’t Even S...,1
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...",0
2,Why the Truth Might Get You Fired,Consortiumnews.com,Consortiumnews.com Why the Truth Might Get You...,1
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Jessica Purkiss 15 Civilians Killed In Single ...,1
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Howard Portnoy Iranian woman jailed for fictio...,1
...,...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...,0
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...",0
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,Michael J. de la Merced and Rachel Abrams Macy...,0
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"Alex Ansary NATO, Russia To Hold Parallel Exer...",1


### Stemming refers to the process of stripping a word down to its root word. For example, 'acting', 'actor' and 'actress' are all stripped to 'act'. This helps as in essence, the impact and functionality of all three words is equivalent to the impact of the word act itself. Stemming will enable the model to give more reasonable and accurate results.

In [10]:
def stemming(inp):
    stemmedinp=re.sub('[^a-zA-Z]',' ', inp)
    stemmedinp=stemmedinp.lower()
    stemmedinp=stemmedinp.split()
    stemmedinp=[PorterStemmer().stem(word) for word in stemmedinp if not word in stopwords]
    stemmedinp=' '.join(stemmedinp)
    return stemmedinp

In [11]:
news['content']=news['content'].apply(stemming)

In [12]:
news['content']

0        darrel lucu hous dem aid didn t comey s letter...
1        daniel j flynn flynn hillari clinton big woman...
2                             consortiumnew com truth fire
3        jessica purkiss civilian kill singl airstrik i...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t trump poster child white...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j la merc rachel abram maci s said rec...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [13]:
X=news['content'].values
Y=news['label'].values
print(X)
print(' ')
print(Y)

['darrel lucu hous dem aid didn t comey s letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth fire' ...
 'michael j la merc rachel abram maci s said receiv takeov approach hudson s bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
 
[1 0 1 ... 0 1 1]


### Vectorizer changes all the strings in our text to a corresponding meaningful numeric value as the Logistic Regression model works only with numeric data. The vectorizer we use is Term Frequency and Inverse Document Frequency Vectorizer (TFIDF). It helps to assign numeric values to words that are repeated a very large number of times in our text input.

In [14]:
vec=TfidfVectorizer()
vec.fit(X)

X=vec.transform(X)

In [15]:
print(X)

  (0, 15615)	0.28955816401922047
  (0, 8874)	0.3696052851985501
  (0, 8594)	0.29695288995239666
  (0, 7657)	0.25194827609967024
  (0, 6970)	0.22235668272295353
  (0, 3989)	0.3015917617273867
  (0, 3772)	0.2750041629677672
  (0, 3580)	0.3658416353803302
  (0, 2939)	0.2509244487438884
  (0, 2463)	0.37372789712937615
  (0, 265)	0.27456502532465643
  (1, 16717)	0.30071745655510157
  (1, 6781)	0.1904660198296849
  (1, 5469)	0.7143299355715573
  (1, 3548)	0.26373768806048464
  (1, 2793)	0.19094574062359204
  (1, 2204)	0.3827320386859759
  (1, 1875)	0.15521974226349364
  (1, 1479)	0.2939891562094648
  (2, 15540)	0.5049376602132613
  (2, 5357)	0.5308172412793459
  (2, 3083)	0.560269092542152
  (2, 2923)	0.3864835825762012
  (3, 13835)	0.3643396661700562
  (3, 11973)	0.49295210164456477
  :	:
  (20797, 14903)	0.31881045786261725
  (20797, 13067)	0.25400168580178045
  (20797, 12289)	0.27894826579153137
  (20797, 12084)	0.2535207421929807
  (20797, 10263)	0.08224225353919856
  (20797, 9551)	0.178

In [16]:
xtrain,xtest,ytrain,ytest=train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [17]:
model=LogisticRegression()

In [ ]:
model.fit(xtrain,ytrain)

In [ ]:
ypred=model.predict(xtest)
test_accuracy=accuracy_score(ypred, ytest)
print("Accuracy Score of Test Data = ",test_accuracy)

### Testing our Logistic Regression Model

In [ ]:
sample=xtest[2000]
pred=model.predict(sample)
if pred==0:
    print("Predicted: Real")
else:
    print("Predicted: Fake")

ans=ytest[2000]
if ans==0:
    print("Actual: Real")
else:
    print("Actual: Fake")